# Overview

This notebook provides a top down view of global equity markets across all sectors and styles based on the cyclically-adjusted price/earnings ratio, or CAPE. 

## What is the CAPE ratio?

In 1998, Robert Shiller and John Campbell published the pathbreaking article “Valuation Ratios and the Long-Run Stock Market Outlook.” A follow-up to some of their earlier work on stock market predictability, it established that long-term stock market returns were not random walks but, rather, could be forecast by a valuation measure called the “cyclically adjusted price–earnings ratio,” or CAPE ratio. Shiller and Campbell calculated the CAPE ratio by dividing a long-term broad-based index of stock market prices and earnings from 1871 by the average of the last 10 years of earnings per share, with earnings and stock prices measured in real terms. They regressed 10-year real stock returns against the CAPE ratio and found that the CAPE ratio is a significant variable that can predict long-run stock returns. The predictability of real stock returns implies that long-term equity returns are mean reverting. In other words, if the CAPE ratio is above (below) its long-run average, the model predicts belowaverage (above-average) real stock returns for the next 10 years. 

*Jeremy J. Siegel (2016) The Shiller CAPE Ratio: A New Look, Financial Analysts Journal, 72:3, 41-50, DOI: 10.2469/faj.v72.n3.1*

# Results

In [16]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
classifications = pd.read_csv('/content/drive/My Drive/nrcapital/data/classification_data.csv', index_col=0).iloc[:, :-2]
grinold_kroner = pd.read_csv('/content/drive/My Drive/nrcapital/data/grinold_kroner_returns.csv', index_col=0)
current_fwd_return_5y_forecast = pd.read_csv('/content/drive/My Drive/nrcapital/data/current_fwd_return_5y_forecast.csv', index_col=0)
etp_prices = pd.read_csv('/content/drive/My Drive/nrcapital/data/etp_prices.csv', index_col=0)

classifications.index.name = 'BENCHMARK_TICKER'
grinold_kroner.index.name = 'BENCHMARK_TICKER'
current_fwd_return_5y_forecast.index.name = 'BENCHMARK_TICKER'

In [8]:
results = pd.read_csv('/content/drive/My Drive/nrcapital/data/equity_etf_posterior_returns.csv')
results.columns = ['BENCHMARK_TICKER', 'ETF_TICKER', 'CORRELATION', 'P_VALUE', 'BENCHMARK_NAME', 'PRIOR_RETURN', 'POSTERIOR_RETURN', 'VIEW']
results = results[['ETF_TICKER', 'CORRELATION', 'P_VALUE', 'PRIOR_RETURN', 'POSTERIOR_RETURN', 'VIEW', 'BENCHMARK_NAME', 'BENCHMARK_TICKER']]
results.ETF_TICKER = results.ETF_TICKER.str.replace(' US Equity', '')
results['ETF_NAME'] = classifications.loc[results.ETF_TICKER.values].NAME.values
results['CLASSIFICATION'] = classifications.loc[results.ETF_TICKER.values].CLASSIFICATION.values
results = results.set_index('ETF_TICKER')
results = pd.merge(results, grinold_kroner, left_on='BENCHMARK_TICKER', right_index=True, how='left')
results = pd.merge(results, current_fwd_return_5y_forecast.FWD_RETURN_5Y_FORECAST, left_on='BENCHMARK_TICKER', right_index=True, how='left')
results.columns

Index(['CORRELATION', 'P_VALUE', 'PRIOR_RETURN', 'POSTERIOR_RETURN', 'VIEW',
       'BENCHMARK_NAME', 'BENCHMARK_TICKER', 'ETF_NAME', 'CLASSIFICATION',
       'LONG_TERM_EARNINGS_YIELD', 'NOMINAL_EARNINGS_GROWTH',
       'REPRICING_RETURN', 'GRINOLD_KRONER_RETURN', 'FWD_RETURN_5Y_FORECAST'],
      dtype='object')

In [9]:
results.loc['DLN']

CORRELATION                                                    0.9639
P_VALUE                                                             0
PRIOR_RETURN                                                 0.097334
POSTERIOR_RETURN                                             0.027536
VIEW                                                           0.0683
BENCHMARK_NAME              Bloomberg Reit Public /Self Storage Index
BENCHMARK_TICKER                                       BBREPBST Index
ETF_NAME                         WisdomTree US LargeCap Dividend Fund
CLASSIFICATION                               U.S. Large-cap Value ETP
LONG_TERM_EARNINGS_YIELD                                       0.0184
NOMINAL_EARNINGS_GROWTH                                        0.0375
REPRICING_RETURN                                              -0.0177
GRINOLD_KRONER_RETURN                                          0.0382
FWD_RETURN_5Y_FORECAST                                         0.0984
Name: DLN, dtype: ob

In [15]:
data = results[['ETF_NAME', 'BENCHMARK_NAME', 'CLASSIFICATION', 'FWD_RETURN_5Y_FORECAST', 'LONG_TERM_EARNINGS_YIELD', 'BENCHMARK_TICKER']]
data.drop_duplicates(subset=['BENCHMARK_TICKER']).sort_values(by='FWD_RETURN_5Y_FORECAST', ascending=False).head(10)

,ETF_NAME,BENCHMARK_NAME,CLASSIFICATION,FWD_RETURN_5Y_FORECAST,LONG_TERM_EARNINGS_YIELD,BENCHMARK_TICKER
ETF_TICKER,,,,,,
EWCO,Invesco S&P 500 Equal Weight Communication Ser...,S&P 600 Consumer Discretionary Sector GICS Lev...,U.S. Large-cap Value ETP,0.2189,0.0781,S6COND Index
FXD,First Trust Consumer Discretionary AlphaDEX Fund,S&P 400 Consumer Discretionary Sector GICS Lev...,U.S. Broad Market Blend ETP,0.1965,0.0567,S4COND Index
GAMR,Wedbush ETFMG Video Game Tech ETF,S&P 500 Apparel & Accessories Sub Industry GIC...,Global Broad Market Growth ETP,0.1940,0.0925,S5APAC Index
XSHQ,Invesco S&P SmallCap Quality ETF,S&P 600 Financials Sector GICS Level 1 Index,U.S. Small-cap Value ETP,0.1745,0.0781,S6FINL Index
CALF,Pacer US Small Cap Cash Cows 100 ETF,S&P Small Cap 600 Index,U.S. Small-cap Value ETP,0.1729,0.0557,SML Index
GSSC,Goldman Sachs ActiveBeta US Small Cap Equity ETF,S&P Small Cap 600 1 Index,U.S. Small-cap Blend ETP,0.1727,0.0557,SMLL1 Index
FYX,First Trust Small Cap Core AlphaDEX Fund,S&P Small Cap 600 Level 3 Index,U.S. Small-cap Value ETP,0.1727,0.0557,SMLL3 Index
AIEQ,AI Powered Equity ETF,S&P Small Cap 600 Growth Index,U.S. Broad Market Growth ETP,0.1720,0.0403,SMLG Index
DGRS,WisdomTree U.S. SmallCap Quality Dividend Grow...,S&P Small Cap 600 Value Index,U.S. Small-cap Value ETP,0.1593,0.0683,SMLV Index


In [6]:
threshold = 10
vif_returns = etp_returns.copy()[lt_pe_.columns]
for i in tqdm(range(vif_returns.shape[1])):
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(vif_returns.values, i) for i in range(vif_returns.shape[1])]
    vif.index = vif_returns.columns
    if (vif.max()[0] > threshold):
        omit = vif.idxmax()
        vif_returns = vif_returns.drop(omit, axis=1)
vif.index.name = 'TICKER'
vif_tickers = list(vif.index)
vif.sort_values(by='VIF Factor')